# Part 1 - WML Federated Learning with MNIST for Admin 

With IBM Federated Learning, you can combine data from multiple sources to train a model from the collective data without having to actually share them. This allows enterprises to train data with other companies without delegating resources for security. Another advantage is the remote data does not have to be centralized in one location, eliminates the needs to move potentially large datasets. This notebook demonstrates how to start Federated Learning with the Python client. For more details setting up Federated Learning, terminology, and running Federated Learning from the UI, see [Federated Learning documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fed-lea.html?audience=wdp).

### Learning Goals

When you complete the Part 1 - WML Federated Learning with MNIST for Admin notebook, you should know how to:

- Load an untrained model
- Create a Remote Training System
- Start a training job

Once you complete this notebook, please open Part 2 - WML Federated Learning with MNIST for Party. 

<div class="alert alert-block alert-info">This notebook is intended to be run by the administrator of the Federated Learning experiment.</div>
</div>

## Table of Contents

- [1. Prequisites](#prequisites)
    - [1.2 Define variables](#var)
    - [1.3 Import libraries](#libraries)
    - [1.3 Obtain Cloud authentication token](#cloud-auth)
- [2. Load the model](#load)
    - [2.1 Create Untrained Model Asset](#model)
    - [2.2 Upload Initial Model](#init)
- [3. Create a Remote Training System](#rts)
    - [3.1 Create Remote Training System Asset](#create-rts)
- [4. Create FL Training Job](#fl-job)
    - [4.1 Get Training Job Status](#status)

<a id = "prequisites"></a>
## 1. Prequisites

Before you proceed, you need to have:

- A Cloud Object Storage bucket. You can create one from your Cloud Object Storage instance in your [IBM Cloud Services](https://cloud.ibm.com/resources).
- A Cloud Object Storage service credential with HMAC enabled. To create one, go to your **Cloud Object Storage < Service Credentials** and click **New credentials**. Give writer access and click **Advanced options** and turn *Include HMAC Credential* to on.
- An IAM API Key. To create a new one, go to [IBM Cloud homepage](https://cloud.ibm.com). In your account, go to **Manage < IAM < API Keys**. Click **Create an IBM Cloud API Key**.

<a id = "var"></a>
### 1.1 Define variables

In [1]:
API_VERSION = "2019-10-25"

WML_SERVICES_HOST = "us-south.ml.cloud.ibm.com" # or "eu-de.ml.cloud.ibm.com"
WML_SERVICES_URL = "https://" + WML_SERVICES_HOST
IAM_TOKEN_URL = "https://iam.cloud.ibm.com/oidc/token"

COS_ENDPOINT_URL = "https://s3.us-south.cloud-object-storage.appdomain.cloud" # See list of endpoints for your location: https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-endpoints

# Get access key ID and secret access key from your HMAC service credentials for COS.
COS_ACCESS_KEY_ID = "xxx" 
COS_SECRET_ACCESS_KEY = "xxx" 

# Name of your COS bucket
COS_BUCKET = "xxx" 

IAM_APIKEY = "xxx"  

# Get this from Manage < IAM < Users, and check the URL. Your user ID should be in the format IBMid-<xxx>.
CLOUD_USERID = "IBMid-123456789" 

PROJECT_ID = "xxx" # Get this by going into your WS project and checking the URL.

<a id = "libraries"></a>
### 1.2 Import libraries

In [2]:
import urllib3
import requests
import json
from string import Template

urllib3.disable_warnings()

<a id = "cloud-auth"></a>
#### Obtain Cloud authentication token

In [3]:
payload = "grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey=" + IAM_APIKEY
token_resp = requests.post(IAM_TOKEN_URL ,
                          headers={"Content-Type": "application/x-www-form-urlencoded"}, 
                          data = payload,
                          verify=True)



In [4]:
print(token_resp)

<Response [200]>


In [ ]:
print(token_resp)
token = "Bearer " + json.loads(token_resp.content.decode("utf-8"))["access_token"]
print("WS token: %s " % token)

<a id = "load"></a>
## 2. Load the model

You need an untrained model asset for Federated Learning to work with. In this tutorial, an untrained Tensorflow 2 Keras model is provided for you. Federated Learning supports Scikit-learn and Tensorflow 2, which are free machine learning packages with tutorials. Additionally IBM docs provide some details on how to configure an untrained model for Federated Learning. See:

- [Scikit-learn model configuration](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-imp-det.html?audience=wdp&context=cpdaas#sklearn-config)
- [Tensorflow 2 model configuration](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fl-imp-det.html?audience=wdp&context=cpdaas#tf-config)

<a id = "model"></a>
### 2.1 Create Untrained Model Asset

Creates an untrained model asset in your project.

In [ ]:
base_model_save_payload = Template("""
{
    "name": "Untrained MNIST Model",
    "type": "tensorflow_2.1 ",
    "software_spec": {
        "name": "default_py3.7"
    },
    "custom": {
        "untrained" : true
    },
    "project_id": "$projectId"
}
""").substitute(projectId = PROJECT_ID)

print ("Model base save payload: %s" % base_model_save_payload)

base_model_save_resp = requests.post(WML_SERVICES_URL + "/ml/v4/models",
                                     headers={"Content-Type": "application/json",
                                              "Authorization": token},
                                     params={"version": API_VERSION},
                                     data=base_model_save_payload,
                                     verify=False)

print("Base model save response " + base_model_save_resp.content.decode("utf-8"))



In [7]:
base_model_id = json.loads(base_model_save_resp.content.decode("utf-8"))["metadata"]["id"]
print("Saved model id: %s" % base_model_id)

Saved model id: 753d765e-de07-4277-ad49-31539a76b228


<a id = "init"></a>
### 2.2 Upload Initial Model

You need to upload the initial model into your project. In this tutorial, an untrained model example is provided and and downloaded from [GitHub](https://github.com/IBMDataScience/sample-notebooks/blob/master/Files/tf_mnist_model.zip).

In [8]:
base_model_content_resp = requests.get("https://github.com/IBMDataScience/sample-notebooks/raw/master/Files/tf_mnist_model.zip",
                                       headers={"Content-Type": "application/octet-stream"})

base_model_content_uri = "/ml/v4/models/"+ base_model_id + "/content"
print("Host URL = " + WML_SERVICES_URL + base_model_content_uri)

base_model_put_resp = requests.put(WML_SERVICES_URL + base_model_content_uri,
                                   headers={"Content-Type": "application/json",
                                            "Authorization": token},
                                   params={"version": API_VERSION,
                                           "project_id": PROJECT_ID,
                                           "content_format": "native"},
                                   data=base_model_content_resp.content,
                                   verify=False)

print("Response status for upload base model  " + base_model_id + " : "+ str(base_model_put_resp.status_code))
print("Create library content: %s"  % base_model_put_resp.content.decode("utf-8"))

Host URL = https://us-south.ml.cloud.ibm.com/ml/v4/models/753d765e-de07-4277-ad49-31539a76b228/content
Response status for upload base model  753d765e-de07-4277-ad49-31539a76b228 : 201
Create library content: {
  "attachment_id": "e822dceb-2c4d-4738-bf1b-533f03ae9fd9",
  "content_format": "native",
  "persisted": true
}


<a id = "rts"></a>
## 3. Create a Remote Training System

Now you will learn to create a Remote Training System (RTS). An RTS handles receiving your multiple parties' call to the aggregator to run the training. 

In [ ]:
training_payload = Template(""" 
{
  "name": "FL Aggregator",
  "tags": [
    {
      "value": "tags_jobs_fl",
      "description": "Sample FL Aggregator"
    }
  ],
  "federated_learning": {
    "model": {
      "spec": {
        "id": "$modelID"
      },
      "type": "tensorflow"
    },
    "fusion_type": "iter_avg",
    "rounds": 5,
    "remote_training" : {
      "quorum": 1.0,
      "remote_training_systems": [ { "id" : "$rts_one", "required" : true  } ]
    },
    "hardware_spec": {
      "name": "XS"
    }
  },
  "training_data_references": [],
  "results_reference": {
    "type": "s3",
    "name": "outputData",
    "connection": {
      "endpoint_url": "$endpoint",
      "access_key_id": "$accessId",
      "secret_access_key": "$secretKey"
    },
    "location": {
      "bucket": "$bucket",
      "path": "."
    }
  },
  "project_id": "$projectId"  
}
""").substitute(modelID = MODEL_ID,
                projectId = PROJECT_ID,
                rts_one = wml_remote_training_system_one_asset_uid,
                endpoint = COS_ENDPOINT_URL,
                accessId = COS_ACCESS_KEY_ID,
                secretKey = COS_SECRET_ACCESS_KEY,
                bucket = COS_BUCKET)

print("Training payload: %s" % training_payload)

training_launch_resp = requests.post(WML_SERVICES_URL + "/ml/v4/trainings", 
                                     headers={"Content-Type": "application/json",
                                              "Authorization": token},
                                     data=training_payload,
                                     verify=False)

print(training_launch_resp)
print("Launch training response %s"  % training_launch_resp.content.decode("utf-8"))
training_id = json.loads(training_launch_resp.content.decode("utf-8"))["metadata"]["id"]
print("Training ID: %s" % training_id)

<a id = "create-rts"></a>
### 3.1 Create Remote Training System Asset

- `allowed_identities` are users permitted to connect to the Federated Learning experiment.  In this tutorial, only your user ID is permitted to connect but you can update the template and add additional users as required.
- An Admin in `remote_admin`. The template for the admin is the same as the user. In this tutorial, a template Admin is created. It is also the same as the user ID, however generally in application, the admin does not have to be one of the users.

In [ ]:
wml_remote_training_system_asset_one_def = Template("""
{
  "name": "Remote Party 1",
  "project_id": "$projectId",
  "description": "Sample Remote Training System",
  "tags": [ "Federated Learning" ],
  "organization": {
    "name": "IBM",
    "region": "US"
  },
  "allowed_identities": [
    {
      "id": "$userID",
      "type": "user"
    }
  ],
  "remote_admin": {
    "id": "$userID",
    "type": "user"
  }
}
""").substitute(userID = CLOUD_USERID,
                projectId = PROJECT_ID)


wml_remote_training_system_one_resp = requests.post(WML_SERVICES_URL + "/ml/v4/remote_training_systems", 
                                                    headers={"Content-Type": "application/json",
                                                             "Authorization": token}, 
                                                    params={"version": API_VERSION,
                                                            "project_id": PROJECT_ID}, 
                                                    data=wml_remote_training_system_asset_one_def, 
                                                    verify=False)

print(wml_remote_training_system_one_resp)
status_json = json.loads(wml_remote_training_system_one_resp.content.decode("utf-8"))
print("Create wml_remote_training_system_one asset response: %s"  % json.dumps(status_json, indent=4))
wml_remote_training_system_one_asset_uid = status_json["metadata"]["id"]
print("WML wml_remote_training_system_one asset uid: %s" % wml_remote_training_system_one_asset_uid)

<a id = "fl-job"></a>
## 4. Create FL Training Job

In this section, you will launch the Federated Learning experiment.

<a id = "status"></a>
### 4.1 Get Training Job Status


In [ ]:
status_full = requests.get(WML_SERVICES_URL + "/ml/v4/trainings/" + training_id,
                           headers={"Content-Type": "application/json",
                                    "Authorization": token},
                           params={"version": API_VERSION,
                                   "project_id": PROJECT_ID},
                           verify=False)

print(status_full)
status_json = json.loads(status_full.content.decode("utf-8"))
print("Full training job status: "+ json.dumps(status_json, indent=4))

### Wait For Aggregator Status To Become Accepting Parties

<div class="alert alert-block alert-warning">Before you proceed further, run the following cell. The output should be `accepting_parties`. If output is `pending`, wait for a few minutes and re-run the <i>previous</i> code cell, with <code>requests.get</code>, and then re-run this code cell again to see the output.</div>
</div>

In [ ]:
json_output = status_json["entity"]["status"]["state"]
print(json_output)

### Get Variables And Paste Into Party Notebook

Run the following cell and copy the output. 

In [ ]:
print("WML_SERVICES_HOST = '%s'" % WML_SERVICES_HOST)
print("IAM_APIKEY = '%s'" % IAM_APIKEY)
print("RTS_ID = '%s'" % wml_remote_training_system_one_asset_uid)
print("TRAINING_ID = '%s'" % (training_id))

As the Admin, you have now launched a Federated Learning experiment. Copy the output from the previous cell. Open Part 2 - WML Federated Learning with MNIST for Party and paste the output into the first code cell.

# <hr>
Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.
 <br><br>
<div style="background:#F5F7FA; height:110px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Love this notebook? </span>
<span style="font-size:15px;color:#152935;float:right;margin-right:40px;">Don't have an account yet?</span><br>
<span style="color:#5A6872;">Share it with your colleagues and help them discover the power of Watson Studio!</span>
<span style="border: 1px solid #3d70b2;padding:8px;float:right;margin-right:40px; color:#3d70b2;"><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
</div>